In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from datetime import datetime

# Extract data from CSV
df1 = pd.read_csv("database.csv")
epoch = datetime(1970, 1, 1)


def mapdateTotime(x):
    try:
        dt = datetime.strptime(x, "%m/%d/%Y")
    except ValueError:
        dt = datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    diff = dt - epoch
    return diff.total_seconds()


df1.Date = df1.Date.apply(mapdateTotime)
col1 = df1[['Date', 'Latitude', 'Longitude', 'Depth']]
col2 = df1['Magnitude']

# Convert to Numpy array
InputX1 = col1.values
InputY1 = col2.values
print(InputX1)

# Min-max Normalization
X1_min = np.amin(InputX1, 0)
X1_max = np.amax(InputX1, 0)
print("Minimum values:", X1_min)
print("Maximum values:", X1_max)
Y1_min = np.amin(InputY1)
Y1_max = np.amax(InputY1)
InputX1_norm = (InputX1 - X1_min) / (X1_max - X1_min)
InputY1_norm = InputY1  # No normalization in output

# Reshape
Xfeatures = 3  # Number of input features
Yfeatures = 1  # Number of input features
samples = 23000  # Number of samples

InputX1_reshape = np.resize(InputX1_norm, (samples, Xfeatures))
InputY1_reshape = np.resize(InputY1_norm, (samples, Yfeatures))

# Training data
batch_size = 2000
InputX1train = InputX1_reshape[0:batch_size, :]
InputY1train = InputY1_reshape[0:batch_size, :]

# Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size + v_size, :]
InputY1v = InputY1_reshape[batch_size:batch_size + v_size, :]

learning_rate = 0.001
training_iterations = 1000
display_iterations = 200

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=(Xfeatures,)),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(Yfeatures)
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.MeanSquaredError())

# Training
model.fit(InputX1train, InputY1train, batch_size=batch_size, epochs=training_iterations, verbose=1)

# Evaluation
train_loss = model.evaluate(InputX1train, InputY1train, verbose=0)
val_loss = model.evaluate(InputX1v, InputY1v, verbose=0)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

# Save the model
model.save("/tmp/earthquake_model.h5")
print("Model saved.")

# Load the model
loaded_model = tf.keras.models.load_model("/tmp/earthquake_model.h5")

[[-1.5768000e+08  1.9246000e+01  1.4561600e+02  1.3160000e+02]
 [-1.5750720e+08  1.8630000e+00  1.2735200e+02  8.0000000e+01]
 [-1.5742080e+08 -2.0579000e+01 -1.7397200e+02  2.0000000e+01]
 ...
 [ 1.4828832e+09  3.6917900e+01  1.4042620e+02  1.0000000e+01]
 [ 1.4829696e+09 -9.0283000e+00  1.1866390e+02  7.9000000e+01]
 [ 1.4830560e+09  3.7397300e+01  1.4141030e+02  1.1940000e+01]]
Minimum values: [-1.57680e+08 -7.70800e+01 -1.79997e+02 -1.10000e+00]
Maximum values: [1.483056e+09 8.600500e+01 1.799980e+02 7.000000e+02]
Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 36.5187
Epoch 2/1000
1/1 [==============================] - 0s 8ms/step - loss: 36.4716
Epoch 3/1000
1/1 [==============================] - 0s 13ms/step - loss: 36.4246
Epoch 4/1000
1/1 [==============================] - 0s 4ms/step - loss: 36.3778
Epoch 5/1000
1/1 [==============================] - 0s 16ms/step - loss: 36.3311
Epoch 6/1000
1/1 [==============================] - 0s 0s/step - loss: 36.2

1/1 [==============================] - 0s 8ms/step - loss: 32.1766
Epoch 96/1000
1/1 [==============================] - 0s 6ms/step - loss: 32.1158
Epoch 97/1000
1/1 [==============================] - 0s 10ms/step - loss: 32.0544
Epoch 98/1000
1/1 [==============================] - 0s 8ms/step - loss: 31.9924
Epoch 99/1000
1/1 [==============================] - 0s 12ms/step - loss: 31.9298
Epoch 100/1000
1/1 [==============================] - 0s 0s/step - loss: 31.8665
Epoch 101/1000
1/1 [==============================] - 0s 17ms/step - loss: 31.8025
Epoch 102/1000
1/1 [==============================] - 0s 0s/step - loss: 31.7381
Epoch 103/1000
1/1 [==============================] - 0s 17ms/step - loss: 31.6732
Epoch 104/1000
1/1 [==============================] - 0s 0s/step - loss: 31.6079
Epoch 105/1000
1/1 [==============================] - 0s 19ms/step - loss: 31.5423
Epoch 106/1000
1/1 [==============================] - 0s 0s/step - loss: 31.4763
Epoch 107/1000
1/1 [==============

1/1 [==============================] - 0s 0s/step - loss: 23.4383
Epoch 196/1000
1/1 [==============================] - 0s 5ms/step - loss: 23.3131
Epoch 197/1000
1/1 [==============================] - 0s 11ms/step - loss: 23.1866
Epoch 198/1000
1/1 [==============================] - 0s 4ms/step - loss: 23.0590
Epoch 199/1000
1/1 [==============================] - 0s 13ms/step - loss: 22.9299
Epoch 200/1000
1/1 [==============================] - 0s 0s/step - loss: 22.7994
Epoch 201/1000
1/1 [==============================] - 0s 14ms/step - loss: 22.6677
Epoch 202/1000
1/1 [==============================] - 0s 0s/step - loss: 22.5347
Epoch 203/1000
1/1 [==============================] - 0s 17ms/step - loss: 22.4002
Epoch 204/1000
1/1 [==============================] - 0s 0s/step - loss: 22.2642
Epoch 205/1000
1/1 [==============================] - 0s 18ms/step - loss: 22.1264
Epoch 206/1000
1/1 [==============================] - 0s 0s/step - loss: 21.9869
Epoch 207/1000
1/1 [===========

1/1 [==============================] - 0s 0s/step - loss: 6.5904
Epoch 296/1000
1/1 [==============================] - 0s 8ms/step - loss: 6.4465
Epoch 297/1000
1/1 [==============================] - 0s 8ms/step - loss: 6.3057
Epoch 298/1000
1/1 [==============================] - 0s 7ms/step - loss: 6.1681
Epoch 299/1000
1/1 [==============================] - 0s 10ms/step - loss: 6.0335
Epoch 300/1000
1/1 [==============================] - 0s 0s/step - loss: 5.9021
Epoch 301/1000
1/1 [==============================] - 0s 13ms/step - loss: 5.7738
Epoch 302/1000
1/1 [==============================] - 0s 0s/step - loss: 5.6488
Epoch 303/1000
1/1 [==============================] - 0s 17ms/step - loss: 5.5269
Epoch 304/1000
1/1 [==============================] - 0s 0s/step - loss: 5.4082
Epoch 305/1000
1/1 [==============================] - 0s 16ms/step - loss: 5.2927
Epoch 306/1000
1/1 [==============================] - 0s 0s/step - loss: 5.1805
Epoch 307/1000
1/1 [========================

1/1 [==============================] - 0s 0s/step - loss: 2.1229
Epoch 397/1000
1/1 [==============================] - 0s 11ms/step - loss: 2.1133
Epoch 398/1000
1/1 [==============================] - 0s 8ms/step - loss: 2.1037
Epoch 399/1000
1/1 [==============================] - 0s 13ms/step - loss: 2.0941
Epoch 400/1000
1/1 [==============================] - 0s 0s/step - loss: 2.0847
Epoch 401/1000
1/1 [==============================] - 0s 16ms/step - loss: 2.0752
Epoch 402/1000
1/1 [==============================] - 0s 0s/step - loss: 2.0659
Epoch 403/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.0566
Epoch 404/1000
1/1 [==============================] - 0s 0s/step - loss: 2.0473
Epoch 405/1000
1/1 [==============================] - 0s 16ms/step - loss: 2.0381
Epoch 406/1000
1/1 [==============================] - 0s 0s/step - loss: 2.0290
Epoch 407/1000
1/1 [==============================] - 0s 13ms/step - loss: 2.0199
Epoch 408/1000
1/1 [======================

1/1 [==============================] - 0s 0s/step - loss: 1.3736
Epoch 498/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.3680
Epoch 499/1000
1/1 [==============================] - 0s 0s/step - loss: 1.3623
Epoch 500/1000
1/1 [==============================] - 0s 11ms/step - loss: 1.3567
Epoch 501/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.3511
Epoch 502/1000
1/1 [==============================] - 0s 0s/step - loss: 1.3455
Epoch 503/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.3399
Epoch 504/1000
1/1 [==============================] - 0s 0s/step - loss: 1.3344
Epoch 505/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.3289
Epoch 506/1000
1/1 [==============================] - 0s 0s/step - loss: 1.3234
Epoch 507/1000
1/1 [==============================] - 0s 0s/step - loss: 1.3180
Epoch 508/1000
1/1 [==============================] - 0s 17ms/step - loss: 1.3126
Epoch 509/1000
1/1 [========================

1/1 [==============================] - 0s 5ms/step - loss: 0.9105
Epoch 599/1000
1/1 [==============================] - 0s 0s/step - loss: 0.9068
Epoch 600/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.9032
Epoch 601/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.8996
Epoch 602/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.8960
Epoch 603/1000
1/1 [==============================] - 0s 0s/step - loss: 0.8924
Epoch 604/1000
1/1 [==============================] - 0s 0s/step - loss: 0.8888
Epoch 605/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.8853
Epoch 606/1000
1/1 [==============================] - 0s 0s/step - loss: 0.8817
Epoch 607/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.8782
Epoch 608/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.8747
Epoch 609/1000
1/1 [==============================] - 0s 0s/step - loss: 0.8712
Epoch 610/1000
1/1 [==========================

1/1 [==============================] - 0s 0s/step - loss: 0.6116
Epoch 700/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.6093
Epoch 701/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6070
Epoch 702/1000
1/1 [==============================] - 0s 0s/step - loss: 0.6047
Epoch 703/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.6024
Epoch 704/1000
1/1 [==============================] - 0s 0s/step - loss: 0.6001
Epoch 705/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.5978
Epoch 706/1000
1/1 [==============================] - 0s 0s/step - loss: 0.5955
Epoch 707/1000
1/1 [==============================] - 0s 0s/step - loss: 0.5933
Epoch 708/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.5910
Epoch 709/1000
1/1 [==============================] - 0s 0s/step - loss: 0.5888
Epoch 710/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.5865
Epoch 711/1000
1/1 [========================

1/1 [==============================] - 0s 0s/step - loss: 0.4255
Epoch 801/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.4241
Epoch 802/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.4227
Epoch 803/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4213
Epoch 804/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.4199
Epoch 805/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4186
Epoch 806/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4172
Epoch 807/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.4159
Epoch 808/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4146
Epoch 809/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.4132
Epoch 810/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4119
Epoch 811/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4106
Epoch 812/1000
1/1 [===========================

1/1 [==============================] - 0s 0s/step - loss: 0.3205
Epoch 902/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3198
Epoch 903/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.3191
Epoch 904/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3184
Epoch 905/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.3176
Epoch 906/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3169
Epoch 907/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3162
Epoch 908/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.3155
Epoch 909/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3148
Epoch 910/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3142
Epoch 911/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.3135
Epoch 912/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3128
Epoch 913/1000
1/1 [===========================

In [2]:
import numpy as np
import tensorflow as tf
from flask import Flask, render_template, request
# Define the Flask app
app = Flask(__name__)
# Home page
@app.route('/')
def index():
    return render_template('index.html', mapdateTotime=mapdateTotime)
# Predict route
@app.route('/detect', methods=['POST'])
def detect():
    latitude = float(request.form['latitude'])
    longitude = float(request.form['longitude'])
    date = request.form['date']
    depth = float(request.form['depth'])

    InputX2 = np.asarray([[latitude, longitude, depth, mapdateTotime(date)]], dtype=np.float32)
    InputX2_norm = (InputX2 - X1_min) / (X1_max - X1_min)
    InputX1test = np.resize(InputX2_norm, (1, Xfeatures))

    # Predict
    output = loaded_model.predict(InputX1test)
    result = output[0][0]  # Extract the predicted value from the output

    return render_template('detect.html', latitude=latitude, longitude=longitude, date=date, depth=depth, result=result, mapdateTotime=mapdateTotime)

# Helpline page
@app.route('/helpline')
def helpline():
    return render_template('helpline.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jul/2023 14:11:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2023 14:11:11] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [06/Jul/2023 14:11:11] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 168ms/step


127.0.0.1 - - [06/Jul/2023 14:11:23] "POST /detect HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2023 14:11:23] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [06/Jul/2023 14:11:23] "GET /static/script.js HTTP/1.1" 404 -
127.0.0.1 - - [06/Jul/2023 14:12:23] "GET /helpline HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2023 14:12:23] "GET /static/style.css HTTP/1.1" 404 -
